### Previous concepts: 
1. **Simple bet For/Against**: all participants can bet for/against, rates are changing after every bet, winners are splitting all bets pool according to their share in winning pool.
- The Problem: there are no insterest to provide more liquidity and looks like rates would not be stable

2. **Liquidity providers**: added liquidity pool that aggregates % from all bets and then distributed between LP according to their shares in LP and deposit time.
- The Problem: it is possible to make a significant change to already accepted bets rate, rates is not fixed

3. **Fixed rate**: all participants possible winning amounts is saved in ledger so their max return is fixed. Rate calculated from current betsForSum/betsAgainsSum ratio that includes participant bet. After all winners withdrawals LP can distribute their pool and share profits proportional to winning pool, losses proportional to losing pool.
- The Problem: multiple small bets are more profitable than one aggregated bet + there are possibility to have positive Ev with this kind of bets [needs to be proved?]

4. **Constant ratio pool**: all participant bets excluded from liquidity pools so La * Lb = L'a * L'b (so this value keept constant before any new liquidity is added)
- The Problem: liquidity providers can have non zero Ev when they are adding liquidity.


### More Alternatives:
The solution to the problem 4 could be in:
* either changing pool ratio when provider adds liquidity
* either changing how profit/loss distributed between providers
* either changing how fixed rate is working

In [8]:
from datetime import datetime
import time


In [16]:
RUN_TIME = int(time.time())
ONE_HOUR = 60*60
DEFAULT_DATE = datetime.strptime('2018-06-30T07:07:32Z', '%Y-%m-%dT%H:%M:%SZ')

INIT_STORAGE = {
    'events': {},
    'betsForLedger': {},
    'betsAgainstLedger': {},
    'providedLiquidityLedger': {},
    'liquidityForBonusLedger': {},
    'liquidityAgainstBonusLedger': {},
    'lastEventId': 0,
    'closeCallEventId': None,
    'measurementStartCallEventId': None
}

EVENT = {
    'currencyPair': 'XTZ-USD',
    'targetDynamics': 1_000_000,
    'betsCloseTime': RUN_TIME + 24*ONE_HOUR,
    'measurePeriod': 12*ONE_HOUR,
    'oracleAddress': 'KT1SUP27JhX24Kvr11oUdWswk7FnCW78ZyUn',
    'liquidityPercent': 10_000,

    'measureStartFee': 200_000,  # who provides it and when?
    'expirationFee': 200_000,    
}


# TODO: should it have interface similar to pytezos.contract? Or it is easier to make proxy after?

class BakingBet:
    """ Fixed rate, bets are excluding from pools.
        Liquidity distribution: profits proportional to winning pool, losses proportional to losing pool
        Problem: Ev of LP is not 0
    """

    def __init__(self, storage):
        self.storage = storage

    def new_event(self, event_params, amount, now):
        last_event_id = self.storage['lastEventId']
        init_event_params = dict(
            createdTime = now,
            targetDynamicsPrecision = 1_000_000,
            measureOracleStartTime = default_date,
            isMeasurementStarted = False,
            startRate = 0,
            isClosed = False,
            closedOracleTime = default_date,
            closedRate = 0,
            closedDynamics = 0,
            isBetsForWin = False,
            betsForSum = 0,
            betsAgainstSum = 0,
            totalLiquidityForBonusSum = 0,
            totalLiquidityAgainstBonusSum = 0,
            totalLiquidityProvided = 0,
            withdrawnLiquidity = 0,
            liquidityPrecision = 1_000_000,
            rewardCallFee = 100_000,
            participants = 0,
            ratioPrecision = 100_000_000,
            betsForWinningPoolSum = 0,
            betsAgainstWinningPoolSum = 0
        )

        init_event_params.update(event_params)
        self.storage['events'][last_event_id] = init_event_params


    def provide_liquidity(self, pl_params, amount, now):
        event = self.storage['events'][pl_params['eventId']]
        event['totalLiquidityProvided'] += amount

        sum_ratio = pl_params['expectedRatioFor'] + pl_params['expectedRatioAgainst']
        for_liquidity = pl_params['expectedRatioFor'] / sum_ratio * amount
        against_liquidity = pl_params['expectedRatioAgainst'] / sum_ratio * amount

        event['betsForSum'] += int(for_liquidity)
        event['betsAgainstSum'] += int(against_liquidity)

        # without time bonus now
        event['totalLiquidityProvided'] += amount
        event['totalLiquidityForBonusSum'] += for_liquidity
        event['totalLiquidityAgainstBonusSum'] += against_liquidity

        # add values to bigmaps
        self.storage['events'][pl_params['eventId']] = event


    def bet(self, bet_params, amount, now, sender):
        event = self.storage['events'][pl_params['eventId']]
        if bet_params['bet'] == 'for':
            # change ratio, calculate winrate, save it
            event['betsForSum'] += amount
        elif bet_params['bet'] == 'against':
            event['betsAgainstSum'] += amount
        else:
            raise Exception('wrong bet params bet')

        self.storage['events'][pl_params['eventId']] = event


model = BakingBet(INIT_STORAGE)
model.new_event(EVENT, amount=400_000, now=RUN_TIME)

pl_params = {'eventId': 0, 'expectedRatioAgainst': 1, 'expectedRatioFor': 6, 'maxSlippage': 100_000}
model.provide_liquidity(pl_params, amount=700, now=RUN_TIME)

bet_params = {'bet': 'for', 'eventId': 0, 'minimalWinAmount': 100}
model.bet(bet_params, amount=100)

In [ ]:
result = contract.bet().with_amount(1_000_000).interpret(
    sender=a, now=RUN_TIME + 6*ONE_HOUR, storage=result.storage)
result.storage['events'][0]['participants']

In [15]:
model.storage

{'events': {0: {'createdTime': 1619790195,
   'targetDynamicsPrecision': 1000000,
   'measureOracleStartTime': datetime.datetime(2018, 6, 30, 7, 7, 32),
   'isMeasurementStarted': False,
   'startRate': 0,
   'isClosed': False,
   'closedOracleTime': datetime.datetime(2018, 6, 30, 7, 7, 32),
   'closedRate': 0,
   'closedDynamics': 0,
   'isBetsForWin': False,
   'betsForSum': 0,
   'betsAgainstSum': 0,
   'totalLiquidityForBonusSum': 0,
   'totalLiquidityAgainstBonusSum': 0,
   'totalLiquidityProvided': 0,
   'withdrawnLiquidity': 0,
   'liquidityPrecision': 1000000,
   'rewardCallFee': 100000,
   'participants': 0,
   'ratioPrecision': 100000000,
   'betsForWinningPoolSum': 0,
   'betsAgainstWinningPoolSum': 0,
   'currencyPair': 'XTZ-USD',
   'targetDynamics': 1000000,
   'betsCloseTime': 1619876595,
   'measurePeriod': 43200,
   'oracleAddress': 'KT1SUP27JhX24Kvr11oUdWswk7FnCW78ZyUn',
   'liquidityPercent': 10000,
   'measureStartFee': 200000,
   'expirationFee': 200000}},
 'betsFo